In [24]:
import concurrent
import ee
import geetools
import geemap
import io
import json
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import requests
import torch
import torchvision
import pandas as pd

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-spencerscw-snow', opt_url='https://earthengine-highvolume.googleapis.com')
SENTINEL = ee.ImageCollection("COPERNICUS/S1_GRD")

In [3]:
station_info = pd.read_json('./stations.json')
station_info.set_index('stationTriplet', inplace=True)
station_info.loc['ABY:CA:SNOW']

stationId                    ABY
stateCode                     CA
networkCode                 SNOW
name                       Abbey
dcoCode                       UN
countyName                Plumas
huc               180201220103.0
elevation                   5760
latitude                  39.955
longitude               -120.538
beginDate       1963-02-01 00:00
endDate         2100-01-01 00:00
shefId                       NaN
dataTimeZone                 NaN
pedonCode                    NaN
Name: ABY:CA:SNOW, dtype: object

In [4]:
depth_2022 = pd.read_json('./depth_2022.json')
depth_2023 = pd.read_json('./depth_2023.json')
depth_2024 = pd.read_json('./depth_2024.json')
depths = []

for year in [depth_2022, depth_2023, depth_2024]:
    for stationTriplet, data in year.itertuples(index=False):
        try:
            latitude = station_info.loc[stationTriplet].latitude
            longitude = station_info.loc[stationTriplet].longitude
            for value in data[0]['values']:
                date = value['date']
                depth = value['value']
                if depth > 0:
                    depths.append({'latitude': latitude, 'longitude': longitude, 'date': date, 'depth': depth})
        except KeyError:
            pass

depth_df = pd.DataFrame(depths)

In [5]:
depth_df

,latitude,longitude,date,depth
0,41.23583,-120.79192,2022-01-01,38
1,41.23583,-120.79192,2022-01-02,37
2,41.23583,-120.79192,2022-01-03,36
3,41.23583,-120.79192,2022-01-04,37
4,41.23583,-120.79192,2022-01-05,35
...,...,...,...,...
524439,40.79492,-106.59544,2024-12-27,39
524440,40.79492,-106.59544,2024-12-28,44
524441,40.79492,-106.59544,2024-12-29,48
524442,40.79492,-106.59544,2024-12-30,47


In [56]:
from torchvision.transforms import v2

sample_coords = [depth_df.iloc[0].longitude, depth_df.iloc[0].latitude]

point = ee.Geometry.Point(sample_coords).buffer(1000)
#point = ee.Geometry.Point(-122.196, 41.411)
sample_image = ee.Image(SENTINEL.filterBounds(point).filterDate(ee.Date('2022-01-01'), ee.Date('2022-01-15')).first().clip(point))

print(sample_image.bandNames().getInfo())
bands = sample_image.bandNames().getInfo()

sample_tensor = torch.tensor(geemap.ee_to_numpy(sample_image, region=point))

padded = torch.nn.functional.pad(sample_tensor[:,:,:2], pad=(0, 1, 0, 0), mode='constant', value=0)

scaler = v2.ToDtype(torch.uint8, scale=True)
floater = v2.ToDtype(torch.float, scale=True)
scaled = floater(scaler(padded))
scaled = torch.permute(scaled, (2, 0, 1))
print(scaled.shape)
torchvision.utils.save_image(scaled, "test.png")

['VV', 'VH', 'angle']


torch.Size([3, 201, 200])


In [21]:
def get_all_images(station_info):
    latitude = station_info["latitude"]
    longitude = station_info["longitude"]
    point = ee.Geometry.Point([longitude, latitude]).buffer(1000)
    all_images = ee.ImageCollection(SENTINEL.filterBounds(point).filterDate(ee.Date('2022-01-01'), ee.Date('2024-12-31')))
    

'stationId'

In [20]:
station_id = 2185

coords = [station_info.iloc[station_id].longitude, station_info.iloc[station_id].latitude]
point = ee.Geometry.Point(sample_coords).buffer(1000)
all_images = ee.ImageCollection(SENTINEL.filterBounds(point).filterDate(ee.Date('2022-01-01'), ee.Date('2024-12-31')))

all_images_list = all_images.toList(all_images.size())
for i in range(0, all_images.size().getInfo(), 50):
    print(ee.Image(all_images_list.get(i)).bandNames().getInfo())

['VV', 'VH', 'angle']


['VV', 'VH', 'angle']


['VV', 'VH', 'angle']


['VV', 'VH', 'angle']


In [41]:
fig, axs = plt.subplots(4, 4)

image_list = all_images.toList(all_images.size()):
for i in range(all_images.size().getInfo()):
    image = ee.Image(image_list.get(i))
    image_tensor = torch.tensor(geemap.ee_to_numpy(image, region=point))
    
    if i % 10 == 0:
        print(i)
    #image.geetools.plot(
    #    bands = [image.bandNames().getInfo()[1]],
    #    ax = axs[i//4, i%4],
    #    region=point,
    #    crs="EPSG:3857",
    #    scale=10,
    #    cmap="viridis",
    #    color="k"
    #)

#plt.show()

0


10


20


30


40


50


60


70


80


90


In [9]:
date = ee.Date(depth_df.iloc[0].date)

date = date.advance(15, 'days')

date.format().getInfo()

'2022-01-16T00:00:00'

In [10]:
from torch.utils.data import Dataset

class SnowDataset(Dataset):
    def __init__(self, depth_df):
        self.depth_df = depth_df
    
    def __len__(self):
        return len(self.depth_df)
    
    def __getitem__(self, idx):
        lat, long, date, depth = self.depth_df.iloc[idx]
        point = ee.Geometry.Point([long, lat]).buffer(1000)
        start_date = ee.Date(date)
        end_date = start_date.advance(15, 'days')
        image = ee.Image(SENTINEL.filterBounds(point).filterDate(start_date, end_date).first().clip(point))
        image_tensor = torch.tensor(geemap.ee_to_numpy(image, region=point))
        
        return image_tensor, depth

In [23]:
from torch.utils.data import DataLoader

snow_dataloader = DataLoader(SnowDataset(depth_df), batch_size = 4, num_workers = 4)

In [24]:
next(iter(snow_dataloader))[0].shape

torch.Size([4, 201, 200, 3])

# Time Log:
## Research, data collection, API practice (total: 8.5 hours)
- 3/3 1.5 hours: Read docs for TorchGeo and Google Earth Engine
- 3/4 2.5 hours: Reread papers from my literature review to find in situ snow measurements
- 3/5 2 hours: Downloaded and cleaned the snow data
- 3/6 2.5 hours: Used Earth Engine API to download and display an S2 image for one target location

## Designing, building, debugging, testing, analyzing, experimenting
- 4/10 2 hours: Wrote the Dataset class and Dataloader to grab images dynamically from Earth Engine for the in-situ observations.
- 4/11 1 hour: More work on the dataloader and creating tensors from the images
- 4/12 2 hours: realized that downloading the images locally is nearly impossible after trying a lot of things